In [ ]:
'''
https://en.wikipedia.org/wiki/Assignment_problem
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html

order of lists is important (eg, student 1 before student 2). in case of draw the preferences of the first list are preferred
student1    [3, 2, 1] -> student 1 prefers topic 3, then 2 and lastly the first topic (C > B > A)

example 1:
weight_of topic1 topic2 topic3
student1    [3, 2, 1], 
student2    [1, 3, 2], 
student3    [2, 1, 3]
->
adjusted (plus one) array indices of best solution: [3 1 2]
S1 gets topic3, S2 gets topic 1, S3 gets topic 2

example 2:
not an issue, if more topics than students
[4,3,1,2], 
[2,4,1,3], 
[2,1,3,4], 
->
adjusted (plus one) array indices of best solution: [3 1 2]

example 3:
if less topics than students, and each topic is assigned more than once:
list each topic accordingly to the number it is assigned. eg, topic1 can be chosen 4 times and topic2 3 times
[1,1,1,1,2,2,2]
[2,2,2,2,1,1,1]
'''

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment

# file_location = r"C:\Users\g21728\Desktop\WS21\Übersicht_Teilnehmer_Master-Thesis.xlsx"
# file_location = r"C:\Users\g21728\Desktop\WS21\Übersicht_Teilnehmer_Bachelor-Thesis.xlsx"
# file_location = r"C:\Users\g21728\Desktop\WS21\Übersicht_Teilnehmer_Masterseminar.xlsx"
file_location = r"C:\Users\g21728\Desktop\WS21\Übersicht_Teilnehmer_Proseminar.xlsx"

# save_location = r"C:\Users\g21728\Desktop\WS21\Übersicht_Teilnehmer_Master-Thesis_assigned.xlsx"
# save_location = r"C:\Users\g21728\Desktop\WS21\Übersicht_Teilnehmer_Bachelor-Thesis_assigned.xlsx"
# save_location = r"C:\Users\g21728\Desktop\WS21\Übersicht_Teilnehmer_Masterseminar_assigned.xlsx"
save_location = r"C:\Users\g21728\Desktop\WS21\Übersicht_Teilnehmer_Proseminar_assigned.xlsx"

df = pd.read_excel(file_location, sheet_name="Tabelle1")

df = df[df["Themen-Präferenzen"].str.contains("zurückgetreten|Rücktritt", regex=True)==False] # drop zurückgetreten students

df["Themen-Präferenzen"] = df["Themen-Präferenzen"].str.replace(' ', '') # remove whitespaces from lists
df["Themen-Präferenzen"] = df["Themen-Präferenzen"].apply(eval) # lists are stored as strings in pandas -> convert with apply(eval)

# convert float columns to int
# df.dtypes # datatype of columns
df["Matrnr"] = df["Matrnr"].astype("Int64") #Int64 can store missing values as NaN

df

In [ ]:
# preferences = [
#     [4,3,1,2], 
#     [2,4,1,3], 
#     [2,1,3,4], 
#     ]

preferences = df['Themen-Präferenzen'].to_list()

cost = np.array(preferences)
row_ind, col_ind = linear_sum_assignment(cost)
adj_ind = [i+1 for i in col_ind]

print(f"Available Topics: {len(preferences[0])}")
print(f"Assigned Topics: {len(col_ind)}")
print(f"Number of students: {df.index[-1] + 1}")
print(f"array indices of best solution: {col_ind}")
print(f"adjusted (plus one) indices of best solution: {adj_ind}\n")


for i,v in enumerate(preferences):
    selected_topic = col_ind[i]+1
    pref_of_selected_topic = v[col_ind[i]]
    print(f"Student {i+1}: Assigned topic {selected_topic} (selected as #{pref_of_selected_topic})")
print(f"\nTotal Sum of preferences (lowest sum possible): {cost[row_ind, col_ind].sum()}\n")

df["scipy.optimize.linear_sum_assignment"] = adj_ind # copy assigned topics to column

# merge assigned topics with Betreuer sheet
df2 = pd.read_excel(file_location, sheet_name="Themen")
# df2

df3 = df.merge(df2, how="left", left_on="scipy.optimize.linear_sum_assignment", right_on="Thema")
df3["Betreuer_x"] = df3["Betreuer_y"]
df3.rename(columns={"Betreuer_x":"Betreuer"}, inplace=True)
df3["Thema DE"] = df3["Titel DE"]
df3.drop(["Titel DE", "Thema", "Betreuer_y"],1, inplace=True)

df3

In [75]:
df3.to_excel(save_location, sheet_name="Final", index=False)

In [ ]:
# adjust weights of specific students
prefe = df['Themen-Präferenzen'].to_list()

# prefer X first students by overweighting their first coice
preferred_number = 1
# prefer first students by adding 
# for i in range(len(prefe)):
#     if i >= preferred_number:
#         x = list(map(lambda x:x+1, prefe[i])) # add 1 to each preference weight for students after preferred students
#         df.loc[i, "Themen-Präferenzen"] = str(x)
#     else:
#         df.loc[i, "Themen-Präferenzen"] = str(prefe[i])

# first choice of preferred students gets -1 weight -> more likely that preferred students get their first choice
for i in range(len(prefe)):
    if i < preferred_number:
        index_min_value = prefe[i].index(min(prefe[i]))
        prefe[i][index_min_value] -= 1 # adjust weight of first choice by minus one
        print(prefe[i])
        df.loc[i, "Themen-Präferenzen"] = str(prefe[i])
    else:
        df.loc[i, "Themen-Präferenzen"] = str(prefe[i])


df["Themen-Präferenzen"] = df["Themen-Präferenzen"].apply(eval) # lists are stored as strings in pandas -> convert with apply(eval)

preferences = df['Themen-Präferenzen'].to_list()
print(preferences)
df